# Lab/ Audio/ Multi-Pitches estimation

Author: Geoffroy Peeters 

Last edit: 2020/11/04

## Introduction

L’objectif de ce TP est de réaliser un module d’estimation de fréquences fondamentales multiples d’un signal de musique (i.e. de déterminer à partir du signal audio les notes de musique jouées).
Il existe de nombreuses approches pour résoudre ce problème. 
Nous étudions ici une technique simple inspirée par les travaux de A. Klapuri [1, 2].

Cette approche consiste à 
- dans un premier temps estimer la fréquence fondamentale dominante
- à en déduire la position et l’amplitude de ses harmoniques sur le spectre 
- et à soustraire la contribution du son correspondant.

Ce principe est ensuite itéré à chaque fois sur le signal soustrait afin d’extraire progressivement l’ensemble des fréquences fondamentales.

Il est ensuite proposé de mettre en place le principe de la **continuité de l’enveloppe spectrale** (spectral smoothness) qui sera appliqué au préalable de la soustraction fréquentielle.

Les performances de ces approches seront testées et comparées sur différents accords composés de notes de piano, de flute et de hautbois.

La figure résume les différentes étapes de l’algorithme.
<IMG src='./_images/TP_multiPitch_schema.png'  width=600/>

In [ ]:
import numpy as np
import scipy.io.wavfile
from scipy.signal import convolve2d
import matplotlib.pyplot as plt
import sys
import tpt_tools
%matplotlib inline

do_student = True

Nous sélectionner le fichier audio à étudier.

In [ ]:
#audioFile = './_audio/A3_piano.wav'# 220 Hz 
#audioFile = './_audio/A4_piano.wav' # 440 Hz
#audioFile = './_audio/E4_oboe.wav' # 329 Hz
#audioFile = './_audio/A5_flute.wav' # 880
#audioFile = './_audio/A3A4_piano.wav' # 220, 440
#audioFile = './_audio/A3A4A5_duo.wav' # 220, 440, 880
audioFile = './_audio/A3C4E4G4_piano.wav' # 220, 261, 329, 392 Hz

In [ ]:
sr_hz, data_v = scipy.io.wavfile.read(audioFile)

fmin_hz = 50
fmax_hz = 900
H = 4
L_sec = 0.1

L_n = np.int(L_sec*sr_hz)
Nfft = 4*tpt_tools.nextpow2(L_n)

import IPython.display as ipd
ipd.Audio(audioFile)

# Estimation de la fréquence fondamentale par la méthode du produit spectral

Nous allons estimer ici la fréquence fondamentale $f_0$ d’un signal discret $x(n)$ échantillonné à la fréquence $sr_{hz}$.
La valeur $f_0$ sera recherchée dans un intervalle $[F_{min},F_{max}]$, avec une **précision** au moins égale à $dF$.
La valeur de $dF$ s'obtient à partir du nombre de points de la DFT et du taux d'échantillonnage $dF = \frac{sr_{hz}}{N_{fft}}$.
Par défaut, nous utiliserons $F_{min}$ = 50 Hz, $F_{max}$ = 900 Hz.

Nous allons utiliser la méthode du **produit spectral**.
Celui-ci est calculé en multipliant $H$ versions compressées du spectre. 

Si nous notons $f_k = \frac{k}{N_{fft}}$ les fréquences discrètes de la DFT, le produit spectral est donné par:
\begin{equation}
    P(f_k) = \prod_{h=1}^{H} |X(h \cdot f_k)|
\end{equation}
Par défaut nous utiliserons $H$ = 4.

Les différentes étapes de ce calcul sont détaillées ci-dessous.

## Calcul du spectre (TFD) du signal

Dans un premier temps, nous calculons la transformée de Fourier discrète (TFD) du signal $x(n)$ sur $N_{fft}$ points.
Nous commençons par multiplier $x(n)$ par une fenêtre de hamming de taille $L_n$ (ceci afin de diminuer la hauteur des lobes secondaires).

La longueur $L_n$ ainsi que le type de cette fenêtre détermine la **résolution fréquentielle**, i.e. le pouvoir de séparer en fréquence deux sinusoides adjacentes.

Le nombre de points de la la DFT, $N_{fft}$, détermine lui la **précision fréquentielle**., i.e. la distance en Hz entre $f_k$ et $f_{k+1}$.
Il faut impérativement utiliser une valeur de $N_{fft}$ supérieure à $L_n$. En pratique $N_{fft}$ est choisi comme une puissance de 2 de manière à pouvoir utiliser l'algorithme de FFT (Fast Fourier Transform). Ceci peut se calculer à l'aide de la fonction ```nextpow2```.

Calculer ensuite la transformée de Fourier Discrète $X(k)$.

In [ ]:
def F_getSpectreFromAudio(x, L_n, Nfft, sr_hz):
    """
    inputs:
        - x: signal
        - L_n: window duration in samples
        - Nfft: fft size
        - sr_hz: sampling rate
    outputs:
        - fftFreq_hz_v (N/2+1,): vector containing the DFT frequencies in Hz
        - fftAmpl_v (N/2+1,): vector containing the DFT amplitudes
    """
    
    if do_student:
        ### START CODE HERE ###    
        ...
        ### END CODE HERE ###
    
    # +++++++++++++++++++++++++++++++
    plt.figure(figsize=(10, 6))
    tpt_tools.F_plot1(fftFreq_hz_v, fftAmpl_v, 'Frequency [Hz]', 'Amplitude [lin]')
    plt.xlim((0,3000))
    # +++++++++++++++++++++++++++++++

    return fftFreq_hz_v, fftAmpl_v

In [ ]:
fftFreq_hz_v, fftAmpl_v = F_getSpectreFromAudio(data_v, L_n, Nfft, sr_hz)

### Test 
Pour le signal audioFile = './_audio/A3C4E4G4_piano.wav' vous devriez obtenir

<IMG SRC='./_images/TP_multiPitch_01.png' width=600/>

## Calcul du produit spectral et recherche du maximum du produit spectral

Le produit spectral $P$ sera codé dans un vecteur de longueur $R$, couvrant l'intervalle $[0, R-1]$.
La fréquence maximale intervenant dans le calcul de $P$ sera donc $H \frac{R-1}{N_{fft}} sr_{hz}$.

En déduire (en fonction de $N_{fft}$ et de $H$) la valeur maximale de $R$ qui garantit de ne pas dépasser la fréquence de Nyquist ($\frac{sr_{hz}}{2}$).
Calculer ensuite $P$ en fonction de $|X|$.

Déterminer les valeurs entières $N_{min}$ et $N_{max}$ qui correspondent à l’intervalle $[F_{min},F_{max}]$ (on veillera à ce que $N_{max}$ reste inférieur à $R$).
Rechercher le maximum de $P$ sur l’intervalle $[N_{min},N_{max}]$, et en déduire la valeur de la fréquence fondamentale $f_0$.

On testera la fonction de détection de la fréquence fondamentale sur des signaux monophoniques (par exemple A4\_piano.wav ou E4\_oboe.wav).

In [ ]:
def F_getF0FromSpectre(fftFreq_hz_v, fftAmpl_v, H, fmin_hz, fmax_hz, Nfft, sr_hz):
    """
    inputs:
        - fftFreq_hz_v (N/2+1,): vector containing the FFT frequencies in Hz
        - fftAmpl_v (N/2+1,): vector containing the FFT amplitude 
        - H: number of times the spectrum is decimated
        - fmin_hz: minimum frequency in Hz to look for F0
        - fmax_hz: maximum frequency in Hz to look for F0
        - Nfft: fft size
        - sr_hz: sampling rate
    outputs:
        - spFreq_hz_v: vector containing the SpectralProduct frequencies in Hz
        - spAmpl_v: vector containing the SpectralProduct amplitudes 
        - f0_hz: estimated F0 in Hz
    """
    
    if do_student:
        ### START CODE HERE ###        
        ...
        ### END CODE HERE ###
    
    # +++++++++++++++++++++++++++++++
    plt.figure(figsize=(10,6))
    tpt_tools.F_plot1(fftFreq_hz_v, fftAmpl_v/np.max(fftAmpl_v), 'Frequency [Hz]', 'Amplitude [lin]')
    plt.plot(spFreq_hz_v, spAmpl_v/np.max(spAmpl_v), 'r')
    plt.xlim((0,3000))
    plt.plot(f0_hz, 1, 'ro')
    # +++++++++++++++++++++++++++++++

    return spFreq_hz_v, spAmpl_v, f0_hz

In [ ]:
spFreq_hz_v, spAmpl_v, f0_hz = F_getF0FromSpectre(fftFreq_hz_v, fftAmpl_v, H, fmin_hz, fmax_hz, Nfft, sr_hz)

### Test 
Pour le signal audioFile = './_audio/A3C4E4G4_piano.wav' vous devriez obtenir

<IMG SRC='./_images/TP_multiPitch_02.png' width=600/>

# Soustraction du son correspondant à la fréquence fondamentale détectée

Dans les cas des signaux polyphoniques, plusieurs spectres harmoniques sont superposés.
Nous allons donc soustraire du spectre total, celui correspondant au spectre harmonique dont nous venons d'avoir estimer la $f_0$.

## Détection des harmoniques

Pour cela, nous devons d'abord connaitre la position des harmoniques du son de fréquence fondamentale $f_0$.
Une procédure simple consiste à rechercher le maximum du spectre autour de chaque **harmonique théorique** de fréquence $f_h = h \cdot f_0$.

L’intervalle de recherche autour de chaque harmonique $f_h$ est donné par $[f_{kmin}:f_{kmax}]$. 
Afin de tenir compte de l'inharmonicité des sons, on pourra choisir $f_{kmin} = (1-\alpha) f_{inharmo}$ et $f_{kmax} = (1+\alpha) f_{inharmo}$ avec $f_{inharmo} = h f_0 \sqrt{1+(h^2-1) \beta}$ où $\beta$ est le coefficient d’inharmonicité (ce qui correspond à un bon modèle d’inharmonicité dans le cas du piano).

On pourra par ailleurs appliquer cette recherche également pour $h$ = 1 afin d’affiner l’estimation de $f_0$.

Quel choix de $\alpha$, $\beta$ vous parait judicieux pour le piano ? pour le hautbois ? 
Ce modèle de recherche des harmoniques vous parait-il judicieux ? Justifier.

**Résponse:**

In [ ]:
def F_getHarmonicsFromF0AndSpectre(fftFreq_hz_v, fftAmpl_v, f0_hz, Nfft, sr_hz):
    """
    inputs:
        - fftFreq_hz_v (N/2+1,): vector containing the FFT frequencies in Hz
        - fftAmpl_v (N/2+1,): vector containing the FFT amplitude 
        - f0_hz: estimated F0 in Hz
        - Nfft: fft size
        - sr_hz: sampling rate
    outputs:
        - harmoFreq_k_v: vector containing the position/index of the Harmonics (position in fftFreq_hz_v or fftAmpl_v)
        - harmoAmpl_v:  vector containing the Harmonic amplitudes
    """
    
    if do_student:
        ### START CODE HERE ###            
        ...
        ### END CODE HERE ###            
    
    # +++++++++++++++++++++++++++++++
    plt.figure(figsize=(10,6))
    tpt_tools.F_plot1(fftFreq_hz_v, fftAmpl_v, 'Frequency [Hz]', 'Amplitude [lin]')
    plt.plot(fftFreq_hz_v, fftAmpl_v, 'b.')
    plt.plot(fftFreq_hz_v[harmoFreq_k_v], harmoAmpl_v, 'ro')
    plt.xlim((0,3000))
    # +++++++++++++++++++++++++++++++

    return harmoFreq_k_v, harmoAmpl_v

In [ ]:
harmoFreq_k_v, harmoAmpl_v = F_getHarmonicsFromF0AndSpectre(fftFreq_hz_v, fftAmpl_v, f0_hz, Nfft, sr_hz)

### Test 
Pour le signal audioFile = './_audio/A3C4E4G4_piano.wav' vous devriez obtenir

<IMG SRC='./_images/TP_multiPitch_03.png' width=600/>

## Suppression des harmoniques

Il s’agit ensuite de supprimer la totalité des harmoniques $f_h$ correspondant à un son donné. 
- Pour cela, on calcul d'abord la largeur théorique d’un pic spectral (celui-ci dépend de la largeur $L_n$ de la fenêtre d’analyse utilisée). 
- Ensuite, pour chaque harmonique $f_h$, on annule le spectre sur cette largeur théorique centrée sur $f_h$. Nous notons $a_h$ et $b_h$ les bornes de cet interval.

En raison de la technique d’estimation de la fréquence fondamentale utilisée, la mise à zéro des harmoniques perturbe les itérations ultérieures de l’algorithme.
Pour cette raison, on remplacera la mise à zéro par le forçage des valeurs du spectre autour de chaque harmonique $f_h$ au minimum du spectre sur cet intervalle ($|X(a_h : b_h)| = \min(|X(a_h : b_h))|$).

## Suppression des harmoniques avec application du principe du "spectral smoothness"

Afin de mieux discerner les sons qui sont en relation harmonique (et notamment les sons à l’octave) il est préférable de ne pas soustraire totalement un son de la mixture, mais d’essayer de ne soustraire que sa contribution au niveau de chaque harmonique.
Pour cela, on utilise le principe de la **continuité de l’enveloppe spectrale ("spectral smoothness")**.

Celui-ci consiste à calculer un spectre harmonique lissé où l’amplitude du spectre au niveau de chaque harmonique $f_h$ est remplacée par la moyenne des amplitudes des harmoniques $f_{h-1}$, $f_h$ et $f_{h+1}$. Cette nouvelle valeur, notée $|\hat{X}(f_h)|$ représente mieux la contribution du son harmonique de fréquence fondamentale $f_0$ que celle issue directement du spectre, notée $|X(f_h)|$. En effet ce dernier peut être le résultat d'une superposition d'harmoniques de différents sons.

Ensuite, la soustraction spectrale consiste à:

– Si $|X(f_h)|$ est inférieure à $|\hat{X}(f_h)|$ nous forçons les valeurs du spectre autour de chaque harmonique $f_h$ au minimum du spectre sur cet intervalle ($|X(a_h : b_h)| = \min(|X(a_h : b_h))|$)

– Si $|X(f_h)|$ est supérieure à $|\hat{X}(f_h)|$ nous forçons les valeurs du spectre autour de chaque harmonique $f_h$ à la différence entre le spectre et le spectre lissé sur cet intervalle $[a_h : b_h]$; pour éviter les valeurs négatives nous clipperons le résultat à un minimum de 0.

In [ ]:
def F_cancelHarmonics(fftFreq_hz_v, fftAmpl_v, harmoFreq_k_v, harmoAmpl_v, L_sec, Nfft, sr_hz):
    """
    inputs:
        - fftFreq_hz_v (N/2+1,): vector containing the FFT frequencies in Hz
        - fftAmpl_v (N/2+1,): vector containing the FFT amplitude 
        - harmoFreq_k_v: vector containing the position/index of the Harmonics (position in fftFreq_hz_v or fftAmpl_v)
        - harmoAmpl_v:  vector containing the Harmonic amplitudes
        - L_sec: window duration in seconds
        - Nfft: fft size
        - sr_hz: sampling rate
    outputs:
        - fftAmplSubtract_v (N/2+1,):  vector containing the FFT amplitude with amplitudes canceled at harmonic positions
    """
 

    Cw = 2.0 # === for a hanning window
    Bw_hz = Cw/(L_sec) # === this is the width at -3dB of the main lobe
    Bw_hz = 3*Bw_hz # === we extend it a bit since we want more than "at -3 dB"
    Bw_k = np.int(Bw_hz/sr_hz*Nfft)
    Bw_k = Bw_k +Bw_k % 2
    if do_student:
        ### START CODE HERE ###
        ...
        ### END CODE HERE ###    
    
    # +++++++++++++++++++++++++++++++
    plt.figure(figsize=(10,6))
    plt.plot(fftFreq_hz_v, fftAmpl_v, 'b-')
    plt.plot(fftFreq_hz_v[harmoFreq_k_v], harmoAmpl_v, 'ro')
    plt.plot(fftFreq_hz_v[harmoFreq_k_v], harmoAmplSmooth_v, 'g+')
    plt.plot(fftFreq_hz_v, fftAmplSubtract_v, 'g-')
    plt.xlim((0,3000))
    # +++++++++++++++++++++++++++++++

    return fftAmplSubtract_v

In [ ]:
fftAmplSubtract_v = F_cancelHarmonics(fftFreq_hz_v, fftAmpl_v, harmoFreq_k_v, harmoAmpl_v, L_sec, Nfft, sr_hz)

### Test 
Pour le signal audioFile = './_audio/A3C4E4G4_piano.wav' vous devriez obtenir

<IMG SRC='./_images/TP_multiPitch_04.png' width=600/>

## Critère d’arrêt

Les étapes précédentes (```F_getF0FromSpectre```,```F_getHarmonicsFromF0AndSpectre```,```F_cancelHarmonics```) sont ensuite itérées tant qu’un certain critère d’arrêt n’est pas vérifié.

Déterminer un critère d’arrêt qui vous permet d’arrêter l’itération lorsque le bon nombre de notes a été trouvé.

On testera l’algorithme sur les différents accords proposés.

In [ ]:
if do_student:
        ### START CODE HERE ###
        ...
        ### END CODE HERE ###    


# Bibliographie

- [1] A.P. Klapuri. Multipitch estimation and sound separation by the spectral smoothness principle. In Procee- dings of IEEE International Conference on Acoustics, Speech and Signal Processing, ICASSP, May 2001.
- [2] A.P. Klapuri. Multiple fundamental frequency estimation by harmonicity and spectral smoothness. IEEE Trans. Speech and Audio Processing, 11(6) :804–816, 2003.